In [39]:
import pandas as pd
import pyodbc

DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}


In [40]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'])
export_cursor = export_conn.cursor()
export_cursor


In [41]:
conn_str_access = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'
)
aec_conn = pyodbc.connect(conn_str_access)


In [42]:
northwind_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=Northwind')
adventureworks_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=AdventureWorks2019')


Queries

In [43]:
AdventureWorks_queries = {
    'AW_Product' : 'SELECT * FROM Production.Product',
    'AW_ProductSubcategory' : 'SELECT * FROM Production.ProductSubcategory',
    'AW_ProductCategory' : 'SELECT * FROM Production.ProductCategory',
    'AW_ProductModel' : 'SELECT * FROM Production.ProductModel',
    'AW_ProductModelIllustration' : 'SELECT * FROM Production.ProductModelIllustration',
    'AW_ProductModelProductDescriptionCulture' : 'SELECT * FROM Production.ProductModelProductDescriptionCulture',
    'AW_ProductPhoto' : 'SELECT * FROM Production.ProductPhoto',
}

Northwind_queries = {
    'NW_Product' : 'SELECT * FROM Products',
    'Categories' : 'SELECT * FROM Categories'
}

AenC_queries = {
    'AC_Product' : 'SELECT * FROM Product'
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, adventureworks_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, aec_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_11136\2765633308.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, adventureworks_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_11136\2765633308.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_11136\2765633308.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[

AdventureWorks

In [44]:
# Merge DataFrames for AdventureWorks product information
AW_Product = pd.merge(AW_Product, AW_ProductSubcategory, left_on='ProductSubcategoryID', right_on='ProductSubcategoryID', suffixes=('', '_subcat'))
AW_Product = pd.merge(AW_Product, AW_ProductCategory, left_on='ProductCategoryID', right_on='ProductCategoryID', suffixes=('', '_cat'))

Northwind

In [45]:
# Merge NW_Product with Categories to get product category information
NW_Product = pd.merge(NW_Product, Categories, on='CategoryID', suffixes=('', '_Category'))

AenC

In [46]:
print(AC_Product)

    id          name        description          prod_size   color quantity  \
0  300     Tee Shirt           Tank Top              Small   White       18   
1  301     Tee Shirt             V-neck             Medium  Orange       39   
2  302     Tee Shirt          Crew Neck  One size fits all   Black       72   
3  400  Baseball Cap         Cotton Cap  One size fits all   Black       92   
4  401  Baseball Cap           Wool cap  One size fits all   White       12   
5  500         Visor        Cloth Visor  One size fits all   White       36   
6  501         Visor      Plastic Visor  One size fits all   Black       28   
7  600    Sweatshirt  Hooded Sweatshirt              Large   Green       39   
8  601    Sweatshirt  Zipped Sweatshirt              Large    Blue       32   
9  700        Shorts      Cotton Shorts             Medium   Black       80   

  unit_price picture_name     Category  
0          9  tshirtw.bmp      Clothes  
1         14  tshirto.bmp      Clothes  
2      

Rename

In [47]:
# Hernoem de kolommen voor Northwind products
NW_Product.rename(columns={
    'ProductID': 'ProductID',
    'ProductName': 'ProductName',
    'ProductDescription': 'ProductDescription',
    'ProductCategory': 'ProductCategory',
    'ProductColor': 'ProductColor',
    'ProductSize': 'ProductSize',
    'ProductQuantity': 'ProductQuantity',
    'QuantityPerUnit': 'QuantityPerUnit',
    'UnitPrice': 'UnitPrice',
    'ListPrice': 'ListPrice',
    'UnitsInStock': 'UnitsInStock'
}, inplace=True)

# Hernoem de kolommen voor AenC products
AC_Product.rename(columns={
    'ProductID': 'ProductID',
    'ProductName': 'ProductName',
    'ProductDescription': 'ProductDescription',
    'ProductCategory': 'ProductCategory',
    'ProductColor': 'ProductColor',
    'ProductSize': 'ProductSize',
    'ProductQuantity': 'ProductQuantity',
    'QuantityPerUnit': 'QuantityPerUnit',
    'UnitPrice': 'UnitPrice',
    'ListPrice': 'ListPrice',
    'UnitsInStock': 'UnitsInStock'
}, inplace=True)

# Hernoem de kolommen voor AdventureWorks products
AW_Product.rename(columns={
    'ProductID': 'ProductID',
    'ProductName': 'ProductName',
    'ProductDescription': 'ProductDescription',
    'ProductCategory': 'ProductCategory',
    'ProductColor': 'ProductColor',
    'ProductSize': 'ProductSize',
    'ProductQuantity': 'ProductQuantity',
    'QuantityPerUnit': 'QuantityPerUnit',
    'UnitPrice': 'UnitPrice',
    'ListPrice': 'ListPrice',
    'UnitsInStock': 'UnitsInStock'
}, inplace=True)

In [48]:
# Voeg de ontbrekende kolommen toe aan elke DataFrame en orden ze in dezelfde volgorde
columns = ['ProductID', 'Source', 'ProductName', 'ProductDescription', 'ProductCategory', 'ProductColor', 'ProductSize', 'ProductQuantity', 
           'QuantityPerUnit', 'UnitPrice', 'ListPrice', 'UnitsInStock']

# Prefixes toevoegen aan ProductID
NW_Product['ProductID'] = NW_Product['ProductID'].apply(lambda x: f'NW_{x}')
AC_Product['ProductID'] = AC_Product['ProductID'].apply(lambda x: f'AC_{x}')
AW_Product['ProductID'] = AW_Product['ProductID'].apply(lambda x: f'AW_{x}')

# Combineer de dataframes
Product = pd.concat([AW_Product, NW_Product, AC_Product], ignore_index=True)

print(Product)


KeyError: 'ProductID'

In [13]:
# Data laden in de nieuwe database (UnitedOutdoors)
for index, row in Employee.iterrows():
    try:
        export_cursor.execute("""
        INSERT INTO Employee (EmployeeID, FirstName, LastName, Region, Country, State, City, PostalCode, Address, DepartmentID, Job_title, Birth_date, Hire_date, Gender, Years_in_Service)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, 
        row.EmployeeID if pd.notnull(row.EmployeeID) else None, 
        row.FirstName if pd.notnull(row.FirstName) else None, 
        row.LastName if pd.notnull(row.LastName) else None, 
        row.Region if pd.notnull(row.Region) else None, 
        row.Country if pd.notnull(row.Country) else None, 
        row.State if pd.notnull(row.State) else None, 
        row.City if pd.notnull(row.City) else None, 
        row.PostalCode if pd.notnull(row.PostalCode) else None, 
        row.Address if pd.notnull(row.Address) else None, 
        int(row.DepartmentID) if pd.notnull(row.DepartmentID) else None, 
        row.Job_title if pd.notnull(row.Job_title) else None, 
        row.Birth_date if pd.notnull(row.Birth_date) else None, 
        row.Hire_date if pd.notnull(row.Hire_date) else None, 
        row.Gender if pd.notnull(row.Gender) else None, 
        row.Years_in_Service if pd.notnull(row.Years_in_Service) else None
        )
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)

export_conn.commit()